In [79]:
class Module:
    def __init__(self, name, c_list=[], a_list=[], d_list=[], group="action"):
        self.name = name #str型
        self.c_list = c_list #リスト型
        self.a_list = a_list #リスト型
        self.d_list = d_list #リスト型
        self.level = 0 #int型
        self.group = group
        
    def act(self):
            print("activated " + self.name + "!")
            #sys.stdout.write(self.name)
            activate = True
            return activate


In [277]:
import copy
class STRIPS:
    def __init__(self, current_state, did_list):
        self.goal_state = set(["clear_B", "B_on_A", "on_table_A","clear_C", "on_table_C","arm_empty"])
        self.current_state = current_state
        self.did_list = did_list
        self.now_level = 0
        self.pick_up_A = Module("pick_up_A", c_list=["clear_A", "on_table_A","arm_empty"], a_list=["holding_A"], d_list=["on_table_A","arm_empty"], group="action")
        self.pick_up_B = Module("pick_up_B", c_list=["clear_B", "on_table_B","arm_empty"], a_list=["holding_B"], d_list=["on_table_B","arm_empty"], group="action")
        self.pick_up_C = Module("pick_up_C", c_list=["clear_C", "on_table_C","arm_empty"], a_list=["holding_C"], d_list=["on_table_C","arm_empty"], group="action")
        self.stack_A_on_B = Module("stack_A_on_B", c_list=["clear_B", "holding_A"], a_list=["arm_empty", "A_on_B"], d_list=["holding_A","clear_B"], group="action")
        self.stack_A_on_C = Module("stack_A_on_C", c_list=["clear_C", "holding_A"], a_list=["arm_empty", "A_on_C"], d_list=["holding_A","clear_C"], group="action")
        self.stack_B_on_A = Module("stack_B_on_A", c_list=["clear_A", "holding_B"], a_list=["arm_empty", "B_on_A"], d_list=["holding_B","clear_A"], group="action")
        self.stack_B_on_C = Module("stack_B_on_C", c_list=["clear_C", "holding_B"], a_list=["arm_empty", "B_on_C"], d_list=["holding_B","clear_C"], group="action")
        self.stack_C_on_A = Module("stack_C_on_A", c_list=["clear_A", "holding_C"], a_list=["arm_empty", "C_on_A"], d_list=["holding_C","clear_A"], group="action")
        self.stack_C_on_B = Module("stack_C_on_B", c_list=["clear_B", "holding_C"], a_list=["arm_empty", "C_on_B"], d_list=["holding_C","clear_B"], group="action")
        self.unstack_A_on_B = Module("unstack_A_on_B", c_list=["arm_empty", 
                                                               "A_on_B", "clear_A"], a_list=["holding_A","clear_B"], d_list=["arm_empty", "A_on_B"], group="action")
        self.unstack_A_on_C = Module("unstack_A_on_C", c_list=["arm_empty", 
                                                               "A_on_C", "clear_A"], a_list=["holding_A","clear_C"], d_list=["arm_empty", "A_on_C"], group="action")
        self.unstack_B_on_A = Module("unstack_B_on_A", c_list=["arm_empty", 
                                                               "B_on_A", "clear_B"], a_list=["holding_B","clear_A"], d_list=["arm_empty", "B_on_A"], group="action")
        self.unstack_B_on_C = Module("unstack_B_on_C", c_list=["arm_empty", 
                                                               "B_on_C", "clear_B"], a_list=["holding_B","clear_C"], d_list=["arm_empty", "B_on_C"], group="action")
        self.unstack_C_on_A = Module("unstack_C_on_A", c_list=["arm_empty", 
                                                               "C_on_A", "clear_C"], a_list=["holding_C","clear_A"], d_list=["arm_empty", "C_on_A"], group="action")
        self.unstack_C_on_B = Module("unstack_C_on_B", c_list=["arm_empty", 
                                                               "C_on_B", "clear_C"], a_list=["holding_C","clear_B"], d_list=["arm_empty", "C_on_B"], group="action")
        self.put_down_A = Module("put_down_A", c_list=["holding_A"], a_list=["on_table_A","arm_empty"], d_list=["holding_A"], group="action")
        self.put_down_B = Module("put_down_B", c_list=["holding_B"], a_list=["on_table_B","arm_empty"], d_list=["holding_B"], group="action")
        self.put_down_C = Module("put_down_C", c_list=["holding_C"], a_list=["on_table_C","arm_empty"], d_list=["holding_C"], group="action")
        



        #self.recognize_bottle = Module("recognize_bottle", c_list=["object_observed"], a_list=["bottle_observed"], d_list=["object_observed"], group="belief")
        #self.recognize_cup = Module("recognize_cup", c_list=["object_observed"], a_list=["cup_observed"], d_list=["object_observed"], group="belief")
        
        self.modules = [self.pick_up_A,self.pick_up_B,self.pick_up_C,
                        self.stack_A_on_B,self.stack_A_on_C,self.stack_B_on_A,self.stack_B_on_C,self.stack_C_on_A,self.stack_C_on_B,
                       self.unstack_A_on_B, self.unstack_A_on_C, self.unstack_B_on_A,self.unstack_B_on_C,self.unstack_C_on_A,self.unstack_C_on_B,
                       self.put_down_A, self.put_down_B,self.put_down_C]
    
    def make_dict(self, c_state):
        module_dict_action = {}
        module_dict_belief = {}
        for module in self.modules:
            if set(module.c_list).issubset(c_state):
                module_dict_action[str(module.name)] = module.level
        #print(module_dict_action)

        return module_dict_action

    
    def selection(self, c_state):
        action = self.make_dict(c_state)
        keys_of_action = list(action.keys())
            
        return keys_of_action
    
    
    def global_loop(self):
        answer = False
        #print(self.now_level)
        for i in range(len(self.current_state[self.now_level])):
            #print(len(self.current_state[self.now_level]))
            action = self.selection(self.current_state[self.now_level][i])
            #print(action)
            if len(action) > 0:
                for j in range(len(action)):
                    self.current_state[self.now_level + 1].append(copy.deepcopy(self.current_state[self.now_level][i]))
                    #print("self.current_state[self.now_level + 1]",self.current_state[self.now_level + 1])
                    #print("action",action[j])
                    #print(self.current_state)
                    for module in self.modules:
                        if module.name == action[j]:
                            self.did_list[self.now_level+1].append(module.name)
                            for state in module.a_list:
                                self.current_state[self.now_level + 1][j].add(state)
                            for state in module.d_list:
                                self.current_state[self.now_level + 1][j].discard(state)
                            if module.name == "stack_B_on_A":
                                print("B",self.current_state[self.now_level + 1][j])
                    #print(self.current_state)
                    if self.goal_state.issubset(self.current_state[self.now_level +1][j]):
                        print("solved!",self.current_state)
                        print("did_list",self.did_list)
                        answer = True
                       
        self.now_level += 1
        #print("did_list",self.did_list)
        #print(self.current_state)
        return answer
                       

In [282]:
def main():
    current_state = [[]  for i in range(30)]
    current_state[0].append(copy.deepcopy(set(["clear_A", "on_table_A","clear_B", "on_table_B","clear_C", "on_table_C","arm_empty"])))
    did_list = [[]  for i in range(30)]
    strips = STRIPS(current_state, did_list)
    for i in range(3):
        a = strips.global_loop()
        if a == True:
            break


In [283]:
main()


B {'on_table_C', 'arm_empty', 'A_on_B', 'B_on_A', 'on_table_B', 'clear_C'}
B {'holding_A', 'holding_C', 'C_on_A', 'arm_empty', 'A_on_B', 'B_on_A', 'on_table_B', 'clear_C'}
B {'holding_A', 'holding_C', 'C_on_A', 'arm_empty', 'A_on_B', 'B_on_A', 'on_table_B', 'clear_C'}
B {'holding_A', 'holding_C', 'C_on_A', 'arm_empty', 'A_on_B', 'B_on_A', 'on_table_B', 'clear_C'}


In [101]:
s = [[],[],[]]

In [109]:
s[1].append( set(["clear_A", "on_table_A","clear_B", "on_table_B","clear_C", "on_table_C","arm_empti"]))

In [110]:
s

[[{'arm_empti',
   'clear_A',
   'clear_B',
   'clear_C',
   'on_table_A',
   'on_table_B',
   'on_table_C'},
  {'arm_empti',
   'clear_A',
   'clear_B',
   'clear_C',
   'on_table_A',
   'on_table_B',
   'on_table_C'},
  {'arm_empti',
   'clear_A',
   'clear_B',
   'clear_C',
   'on_table_A',
   'on_table_B',
   'on_table_C'}],
 [{'arm_empti',
   'clear_A',
   'clear_B',
   'clear_C',
   'on_table_A',
   'on_table_B',
   'on_table_C'}],
 []]

In [24]:
s[0][2]

{'arm_empti',
 'clear_A',
 'clear_B',
 'clear_C',
 'on_table_A',
 'on_table_B',
 'on_table_C'}